In [1]:
from pymongo import MongoClient

from gensim import corpora, models, similarities, matutils

from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from collections import Counter
import numpy as np
import pandas as pd
import string

In [2]:
client = MongoClient()
db = client.steamreviews

In [86]:
db.list_collection_names()

['steamreviews', 'negreviews', 'roguelikes']

In [4]:
db.negreviews.find_one()

{'_id': ObjectId('5e5069a2c5cc61c29a4a0336'),
 'recommendationid': '63896597',
 'author': {'steamid': '76561198057375143',
  'num_games_owned': 2,
  'num_reviews': 1,
  'playtime_forever': 1468,
  'playtime_last_two_weeks': 0,
  'last_played': 1572227072},
 'language': 'english',
 'review': 'Relaxing fun',
 'timestamp_created': 1582321211,
 'timestamp_updated': 1582321211,
 'voted_up': True,
 'votes_up': 0,
 'votes_funny': 0,
 'weighted_vote_score': 0,
 'comment_count': 0,
 'steam_purchase': True,
 'received_for_free': False,
 'written_during_early_access': False}

In [5]:
db.negreviews.count()

<ipython-input-5-2789f2b9ff57>:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  db.negreviews.count()


243740

In [40]:
reviewlist = list(db.negreviews.find({'voted_up':False},{'_id':0}))

In [92]:
revlstRL = list(db.roguelikes.find({'voted_up':False},{'_id':0}))

In [91]:
len(revlstRL)

347648

In [89]:
len(revlstRL)

26270

In [35]:
len(reviewlist)

71179

In [37]:
reviewlist

[{'recommendationid': '63875178',
  'author': {'steamid': '76561198185897443',
   'num_games_owned': 17,
   'num_reviews': 3,
   'playtime_forever': 63,
   'playtime_last_two_weeks': 63,
   'last_played': 1582289607},
  'language': 'english',
  'review': "Runs Terribly, I played it on PS4 a while back and bought it on PC because I prefer PC gaming but doesn't even run above 15fps !!! Before anybody even asks it's not a hardware problem since my specs are (even when running at minimal graphics settings)\nGeForce GTX 960\n12GB DDR4 RAM\n3.5GHZ CPU\n",
  'timestamp_created': 1582289925,
  'timestamp_updated': 1582289925,
  'voted_up': False,
  'votes_up': 1,
  'votes_funny': 3,
  'weighted_vote_score': '0.33109620213508606',
  'comment_count': 5,
  'steam_purchase': True,
  'received_for_free': False,
  'written_during_early_access': False},
 {'recommendationid': '63845518',
  'author': {'steamid': '76561198055961210',
   'num_games_owned': 300,
   'num_reviews': 2,
   'playtime_forever':

In [54]:
reviewsonly = []
for i in range(len(list(reviewlist[:][:]))):
    reviewsonly.append(list(reviewlist[:][i].values())[3])

In [55]:
reviewsonly

["Runs Terribly, I played it on PS4 a while back and bought it on PC because I prefer PC gaming but doesn't even run above 15fps !!! Before anybody even asks it's not a hardware problem since my specs are (even when running at minimal graphics settings)\nGeForce GTX 960\n12GB DDR4 RAM\n3.5GHZ CPU\n",
 'Ignore the "internet historian" posts. Most of them don\'t have anything useful.\n\n\nThe game has improved? I guess? But it\'s been 3-4 years *since release* now. it\'s nowhere near good enough. \n\nCrashed in the first few minutes just moving the camera around.\nMenus are clunky at best\nTexture loading was terrible.\nHitboxes were terrible. \nGameplay was boring.\n\nI was told this game gets boring and grindy and starts crashing at certain points after playing for several hours already. But the first couple hours already achieved those things.\n\nIf it actually fixes all the crashing and graphic issues, it might be worth half price as an exploration game. It *still* isn\'t right now,,

In [56]:
reviewlist

[{'recommendationid': '63875178',
  'author': {'steamid': '76561198185897443',
   'num_games_owned': 17,
   'num_reviews': 3,
   'playtime_forever': 63,
   'playtime_last_two_weeks': 63,
   'last_played': 1582289607},
  'language': 'english',
  'review': "Runs Terribly, I played it on PS4 a while back and bought it on PC because I prefer PC gaming but doesn't even run above 15fps !!! Before anybody even asks it's not a hardware problem since my specs are (even when running at minimal graphics settings)\nGeForce GTX 960\n12GB DDR4 RAM\n3.5GHZ CPU\n",
  'timestamp_created': 1582289925,
  'timestamp_updated': 1582289925,
  'voted_up': False,
  'votes_up': 1,
  'votes_funny': 3,
  'weighted_vote_score': '0.33109620213508606',
  'comment_count': 5,
  'steam_purchase': True,
  'received_for_free': False,
  'written_during_early_access': False},
 {'recommendationid': '63845518',
  'author': {'steamid': '76561198055961210',
   'num_games_owned': 300,
   'num_reviews': 2,
   'playtime_forever':

In [57]:
list(reviewlist[:][0].values())[3]

"Runs Terribly, I played it on PS4 a while back and bought it on PC because I prefer PC gaming but doesn't even run above 15fps !!! Before anybody even asks it's not a hardware problem since my specs are (even when running at minimal graphics settings)\nGeForce GTX 960\n12GB DDR4 RAM\n3.5GHZ CPU\n"

In [5]:
#reviewlist = pd.read_csv('negreviewsonly.csv')

In [8]:
#pd.DataFrame(reviewsonly).to_csv('reviewsonly.csv')

In [64]:
reviewlist[:][1].values()

dict_values(['63845518', {'steamid': '76561198055961210', 'num_games_owned': 300, 'num_reviews': 2, 'playtime_forever': 44, 'playtime_last_two_weeks': 44, 'last_played': 1582174423}, 'english', 'Ignore the "internet historian" posts. Most of them don\'t have anything useful.\n\n\nThe game has improved? I guess? But it\'s been 3-4 years *since release* now. it\'s nowhere near good enough. \n\nCrashed in the first few minutes just moving the camera around.\nMenus are clunky at best\nTexture loading was terrible.\nHitboxes were terrible. \nGameplay was boring.\n\nI was told this game gets boring and grindy and starts crashing at certain points after playing for several hours already. But the first couple hours already achieved those things.\n\nIf it actually fixes all the crashing and graphic issues, it might be worth half price as an exploration game. It *still* isn\'t right now,, and that\'s just sad.', 1582234592, 1582234592, False, 1, 5, '0.246611356735229492', 0, True, False, False])

In [65]:
import re
words = re.split(r'\W+', list(reviewlist[:][0].values())[3])
print(words[:100])

['Runs', 'Terribly', 'I', 'played', 'it', 'on', 'PS4', 'a', 'while', 'back', 'and', 'bought', 'it', 'on', 'PC', 'because', 'I', 'prefer', 'PC', 'gaming', 'but', 'doesn', 't', 'even', 'run', 'above', '15fps', 'Before', 'anybody', 'even', 'asks', 'it', 's', 'not', 'a', 'hardware', 'problem', 'since', 'my', 'specs', 'are', 'even', 'when', 'running', 'at', 'minimal', 'graphics', 'settings', 'GeForce', 'GTX', '960', '12GB', 'DDR4', 'RAM', '3', '5GHZ', 'CPU', '']


In [66]:
words = list(reviewlist[:][0].values())[3].split()
# convert to lower case
words = [word.lower() for word in words]
print(words[:100])

['runs', 'terribly,', 'i', 'played', 'it', 'on', 'ps4', 'a', 'while', 'back', 'and', 'bought', 'it', 'on', 'pc', 'because', 'i', 'prefer', 'pc', 'gaming', 'but', "doesn't", 'even', 'run', 'above', '15fps', '!!!', 'before', 'anybody', 'even', 'asks', "it's", 'not', 'a', 'hardware', 'problem', 'since', 'my', 'specs', 'are', '(even', 'when', 'running', 'at', 'minimal', 'graphics', 'settings)', 'geforce', 'gtx', '960', '12gb', 'ddr4', 'ram', '3.5ghz', 'cpu']


In [20]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\benny\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [32]:
len(reviewlist['0'])

257207

In [81]:
from nltk.tokenize import word_tokenize
tokens = word_tokenize(reviewlist['0'][1])
print(tokens[:100])

TypeError: list indices must be integers or slices, not str

In [82]:
wordlist = []
for i in range(len(list(reviewlist))):
    tokens = word_tokenize(str(list(reviewlist[:][i].values())[3]))
    for word in tokens:
        wordlist.append(word)
wlset = list(set(wordlist))

In [78]:
len(list(reviewlist))

71179

In [83]:
len(wlset)

106955

In [84]:
for i in range(len(wlset)):
    wlset[i] = wlset[i].lower()
len(list(set(wlset)))

87764

In [85]:
for i in range(len(wlset)):
    wlset[i] = wlset[i].translate(str.maketrans('', '', string.punctuation))
len(list(set(wlset)))

76798

In [82]:
#pd.DataFrame(wlset).to_csv('wordset5.csv')


In [68]:
type(reviewlist['0'][0])

str

In [71]:
#def make_unicode(input):
#    if type(input) != unicode:
#        input =  input.decode('utf-8')
#    return input

In [76]:
reviewlist['0'][3].encode(encoding='UTF-8',errors='strict')

b'good for anybody with dad issues'

In [78]:
unireviews = []
for item in reviewlist['0']:
    unireviews.append(str(item).encode(encoding='UTF-8',errors='strict'))

In [79]:
unireviews

[b'\xe9\x9d\x9e\xe5\xb8\xb8\xe5\xa5\xbd\xe7\x8e\xa9\xe9\x9d\x9e\xe5\xb8\xb8\xe7\x88\xbd',
 b"There are so many things to like about Hades: the beautiful environments; the silky smooth animation of the characters; the 'just one more go' roguelite elements; the Greek mythology; the amount of content currently in the Early Access version; the permanent upgrade system; the quirky character dialogue...\n\nIf you like roguelites, action games, or indie games in general, then you owe it to yourself to get Hades.",
 b'wonderful game',
 b'good for anybody with dad issues',
 b'\xe7\x8e\xa9\xe5\xbe\x97\xe6\x84\x9b\xe4\xb8\x8d\xe9\x87\x8b\xe6\x89\x8b \xe5\x81\x9c\xe4\xb8\x8d\xe4\xb8\x8b\xe4\xbe\x86\xe4\xba\x86',
 b'\xe5\x9c\xa8\xe6\x9c\x89\xe9\x99\x90\xe7\x9a\x84\xe6\xb8\xb8\xe6\x88\x8f\xe5\x86\x85\xe5\xae\xb9\xe5\x86\x85\xe5\x87\xa0\xe4\xb9\x8e\xe5\x81\x9a\xe5\x88\xb0\xe4\xba\x86\xe5\xae\x8c\xe7\xbe\x8e\xef\xbc\x8c\xe6\x8c\x91\xe4\xb8\x8d\xe5\x87\xba\xe5\x88\xba\xe7\x9a\x84rougelike\xe6\xb8\xb8\x

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [80]:
cv_tfidf = TfidfVectorizer(stop_words='english')
X_tfidf = cv_tfidf.fit_transform(unireviews).toarray()
df_tfidf = pd.DataFrame(X_tfidf, columns= X_tfidf.get_feature_names())
df_tfidf

MemoryError: Unable to allocate 244. GiB for an array with shape (257207, 127423) and data type float64

In [64]:
reviewlist

,Unnamed: 0,0
0,0,非常好玩非常爽
1,1,There are so many things to like about Hades: ...
2,2,wonderful game
3,3,good for anybody with dad issues
4,4,玩得愛不釋手 停不下來了
...,...,...
257202,257202,Harvest Moon on steroids.\n\nMade by a single ...
257203,257203,Simply amazing. \n\nI've waited for this game ...
257204,257204,I have been waiting for this for a very long t...
257205,257205,If you love Animal Crossing or Harvest Moon th...


In [24]:
for i in range(len(tokens)):
    tokens[i]=tokens[i].lower()

In [6]:
class My_CountVectorizer:
    
    def fit_vocab(self, corpus):
        
        # collect vocabulary word counts
        self.counter = Counter()
        for doc in corpus:
            self.counter.update(doc.split(' '))
        
        # map words to columns in descending order of frequency
        self.feature_map = {word : i for i, (word, count) 
                            in enumerate(self.counter.most_common())}
           
    def transform_corpus(self, corpus):
        
        vectors = []
        
        # fill doc rows by iterating through words and 
        # accumulating counts to term columns
        for doc in corpus:
            vector = np.zeros(len(self.feature_map))
            for word in doc.split(' '):
                vector[self.feature_map[word]] += 1
            vectors.append(vector)
        
        # document-term matrix with word column names
        word_df = pd.DataFrame(vectors, columns=self.feature_map.keys())
        return word_df

In [11]:
cv = My_CountVectorizer()
cv.fit_vocab(reviewlist['0'])
asdf = cv.transform_corpus(reviewlist['0'])

AttributeError: 'float' object has no attribute 'split'

In [ ]:
count_vectorizer = CountVectorizer(ngram_range=(1, 2),  
                                   stop_words='english', token_pattern="\\b[a-z][a-z]+\\b")